# 14 неделя: Численные методы

## Методы численного интегрирования и дифференцирования.

На прошлом занятии мы увидели пользу от рядов и полиномов для численных расчетов.

Тема этого занятия - методы численного интегрирования и дифференцирования.
Это достаточно стандартные задачи, очень востребованные в деятельности физика.

<b>Литература для чтения:</b>
* Хамминг - Численные методы для физика и инженера.
* Ильина, Силаев - Численные методы для физиков-теоретиков, в 2-х тт.

### 1) Интегрирование.


Всем известный определённый интеграл в конечных пределах:
$$
J(a,b) = \int \limits_a^b f(x) dx = F(a) - F(b)
$$

Объясним пользу от численного интегрирования.
Прежде всего надо сказать, что отнюдь не все интегралы можно рассчитать аналитически.
Так, например, т.н. функция ошибок
$$
\mathrm{erf}(x) = \frac{2}{\sqrt{\pi}} \int \limits_0^x e^{-t^2} dt,
$$

интегральный синус
$$
\mathrm{Si}(x) = \int \limits_0^x \frac{\sin(t)}{t} dt,
$$

и многие другие не имеют ответа в виде удобных замкнутых формул. Они рассчитываются только численно, либо при помощи различных вспомогательных представлений (см. Дополнительные материалы).

Более того, даже если аналитическая формула и существует, она может быть слишком громоздкой и поэтому неудобной  для использования.

И самое главное -- абсолютно правильный ответ в виде формулы часто не нужен. Например, при планировании или обработке результатов вашего эксперимента требуется только получить конкретное число,
которое будет являться искомым параметром исследуемого образца или явления: тепловыделение реактора, процентное содержание элемента, эффективность установки и т.д.

Т.е. вперёд выходит соображение практичности, когда достаточно определить 
конкретное значение искомой величины с какой-то допустимой погрешностью.
В таком случае снова помогают численные подходы.

### Сеточные методы интегрирования

Вспомним формулу Римана для определенного интеграла:
$$
I = \int \limits_a^b f(x) dx = \lim_{\Delta x \to 0}
\sum \limits_{i=0}^{n-1} f(x_i) \Delta x , 
\hspace{10mm} x_i = a + \Delta x \cdot i
$$
где промежуток $[a,b]$ оси $х$ делится на $n$ частей длиной $\Delta x = \frac{a-b}{n}$.

Заметим, что эта формула вполне годится для приближенного расчёта на компьютере.
Современный процессор обладает производительностью порядка $10^{11}$ расчетных операций в секунду (FLOPS), поэтому даже миллион слагаемых просуммируется за малую долю секунды
(конечно, если подынтегральное выражение достаточно простое).

Взяв достаточно малый шаг, можно по этой формуле получить число, очень близкое к искомому значению интеграла.

Это - так называемый <b>метод прямоугольников</b> на равномерной сетке.
Он имеет простую графическую аналогию: интеграл функции $f(x)$ на промежутке $x$ от $а$ до $b$ равен площади под кривой, описываемой этой функцией.
Если мы заменим фигуру под графиком функции множеством прямоугольников $h \times f_i$,
то сумма их площадей будет приближённо равна значению искомого интеграла:
$$
\int_{a}^{b} f(x)dx
\simeq h\sum_{i=1}^n f(x_{i-1})
\simeq h\sum_{i=1}^n f\left(x_{i-1}+\frac{h}{2} \right)
\simeq h\sum_{i=1}^n f(x_{i}),
\hspace{10mm} h=\frac{b-a}{n}
$$

Положение точек $x$ внутри отрезков, где рассчитывается значение функции (слева, по центру, справа) выбирается из соображений удобства. 

Метод прямоугольников имеет первый порядок точности. 
Это означает, что ошибка пропорциональна шагу.
Соответственно, чем меньше шаг, тем меньшую ошибку мы получим.

<b>Пример:</b> рассчитаем интеграл 
$$
\int \limits_0^1 \frac{1}{x^2+1} dx = arctan(1) = \frac{\pi}{4}
$$

In [ ]:
import numpy as np

def f(x):
    return 1/(x**2 + 1)

def test_arctan(b, n):
    h = b/n
    theor = np.arctan(b)
    s = 0
    for i in range(n):
        x = h*i
        s = s + h*f(x)
    print(" n=", n, "\t h= ", h, "\t погрешность: ", s-theor)

print("atan(1)=", np.arctan(1))
test_arctan(1, 100)
test_arctan(1, 1000)
test_arctan(1, 10000)
test_arctan(1, 100000)


atan(1)= 0.7853981633974483
 n= 100 	 h=  0.01 	 погрешность:  0.002495833333333475
 n= 1000 	 h=  0.001 	 погрешность:  0.00024995833333218975
 n= 10000 	 h=  0.0001 	 погрешность:  2.4999583330909125e-05
 n= 100000 	 h=  1e-05 	 погрешность:  2.499995840454794e-06


Как видно, погрешность расчёта действительно пропорциональна $h$.

Существуют сеточные методы с более высокой точностью.

Так, например, следующий по точности <b>метод трапеций</b> приближает интегрируемую функцию уже не константой, а линейной функцией, "натянутой" между соседними значениями функции, и теперь суммируются площади трапеций:

\begin{equation}
I \simeq
 h\sum_{i=0}^{n-1} \frac{f(x_i)+f(x_{i+1})}{2}  =
h\left(\frac{f(a )+f(b)}{2} + \sum_{i=1}^{n-1} f(x_i) \right).
\end{equation}

Обратите внимание на пределы индекса суммирования: крайние точки $x_0=a$ и $x_n=b$  добавляются отдельно, т.к. имеют другой коэффициент в отличие от точек внутри.

Здесь погрешность расчёта пропорциональна $h^2$.




Также заслуживает внимания <b>метод Симпсона</b>, который приближает интегрируемую функцию параболой, т.е. использует ещё больше членов разложения в ряд Тейлора и поэтому должен давать ещё более точный ответ:

\begin{equation}
I 
\simeq \frac{h}{3} \sum \limits_{i=0}^{n/2-1} \Bigl[
f(x_{2i}) + 4 f(x_{2i+1}) + f(x_{2i+2}) 
\Bigr]
= \frac{h}{3} \Biggl\{
    - f(a) + f(b) + \sum \limits_{i=1}^{n/2-1} \bigl[
        2 f(x_{2i}) + 4 f(x_{2i+1}) 
    \bigr]
\Biggr\}.
\end{equation}

Обратите внимание на меняющиеся коэффициенты при значениях функции в точках с чётными и нечётными индексами.

Подводя итог, приведём таблицу по этим самым известным методам интегрирования на равномерной сетке:
    
|Метод| Число точек| Коэффициенты|  Точность|
|:-|:-:|:-:|---|
|Прямоугольники|1|h|$\sim h^1$ |
|Трапеции|2| $ \frac{h}{2}\begin{bmatrix}1 & 1 \end{bmatrix} $|$\sim h^2$|
|Симпсона|3|$ \frac{h}{3}\begin{bmatrix}1 & 4 & 1 \end{bmatrix} $| $\sim h^3$|

Как видно, для повышения точности расчёта не обязательно
брать больше слагаемых ($10^5$, $10^6$ и так далее)
-- можно просто взять метод с более высоким порядком точности (больше информации в Дополнительных материалах к занятию).

<b>Код:</b> сравним точность расчёта того же интеграла для трёх методов из таблицы.


In [ ]:
def test_arctan_all(b, n):
    h = b/n
    theor = np.arctan(b)
    s1 = 0
    for i in range(n):
        x = h*(i+0.5)
        s1 = s1 + f(x)
    s1 = h*s1
    
    s2 = 0
    for i in range(1,n):
        x = h*i
        s2 = s2 + f(x)
    s2 = h* (s2 + (f(0) + f(b))/2)
    
    s3 = 0
    for i in range(n//2):
        x = h*i*2
        s3 = s3 + f(x) + 4*f(x+h) + f(x+h*2)
    s3 = h*s3/3
    
    print(n, "\t", s1-theor, "\t",
         s2-theor, "\t", s3-theor)
    
print("n \t метод прямоугольников \t трапеций \t\t\t Симпсона")    
test_arctan_all(1, 20)
test_arctan_all(1, 100)
test_arctan_all(1, 10000)

n 	 метод прямоугольников 	 трапеций 			 Симпсона
20 	 5.208332582529174e-05 	 -0.00010416665891610499 	 -1.550017891815969e-10
100 	 2.083333333069426e-06 	 -4.1666666660278295e-06 	 -9.658940314238862e-15
10000 	 2.083352379500525e-10 	 -4.166632594504449e-10 	 2.7755575615628914e-15


Как видно, метод Симпсона очень точный: в данном случае даже очень грубого разбиения отрезка на 20 частей достаточно, чтобы получить точность в 10 знаков, для чего методу прямоугольников потребуется 10000 слагаемых, т.е. в 500 раз больше.
Кажется, что разница невелика, т.к. в обоих случаях расчёт интеграла всё равно занимает малую долю секунды. Но в случае профессионального применения интегралы нужно считать постоянно и ускорение в 500 раз будет очень кстати.

<b>Материал для дополнительного чтения:</b>
* метод Симпсона "3/8" (4-й порядок точности)
* метод Буля (5-й порядок точности)
* метод Ромберга (метод трапеций при нескольких значениях $h$ и экстраполяция $h \to 0$).
